In [4]:
#Import required modules
import geopandas as gpd
import fiona
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from shapely.geometry import Polygon
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [ ]:
#Read in watershed boundary dataset at HU6 level
hucs = gpd.read_file("Data/Sources/WBD_National_GDB.gdb", driver='FileGDB', layer='WBDHU6')

#Read in counties shapefiles
counties = gpd.read_file("Data/Sources/tl_2017_us_county")

world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
usa = world[world['iso_a3'] == 'USA']

#Clip boundaries to continental United States
usac = gpd.clip(usa, Polygon([(-130, 20), (-130, 55), (-60, 55), (-60, 20), (-130, 20)]))

hucs_clipped = hucs.copy()
for i in hucs_clipped.index:
    try:
        if gpd.clip(hucs_clipped.loc[[i]], usac).shape[0] == 0:
            hucs_clipped.drop(i, inplace = True) #Drop HUC6 codes outside of the continental US
        else:
            hucs_clipped.loc[[i]] = gpd.clip(hucs_clipped.loc[[i]], usac)
    except:
        pass
counties_clipped = gpd.clip(counties, usac)

In [ ]:
#Plot outlines of counties
base = usac.boundary.plot(figsize = (14, 12), color = 'gray', linewidth = 0.1)
counties_clipped.boundary.plot(ax = base, color = 'k', linewidth = 0.1);
plt.axis('off')
plt.tight_layout();
plt.savefig('Plots/Maps/USA_counties.jpg')

#Plot outlines of HU6 watersheds
base = usac.boundary.plot(figsize = (14, 12), color = 'gray', linewidth = 0.1)
counties_clipped.boundary.plot(ax = base, color = 'k', linewidth = 0.1);
hucs_clipped.boundary.plot(ax = base, color = 'c', linewidth = 0.2)
plt.axis('off')
plt.tight_layout();
plt.savefig('Plots/Maps/USA_counties_watersheds.jpg')

In [ ]:
#Use Geopandas overlay feature to quantify intersections between each county and watershed
hc_intersect = gpd.overlay(counties, hucs, how = 'intersection')

hc_intersect = hc_intersect[['STATEFP', 'COUNTYFP',  'HUC6', 'geometry']]

hc_intersect['intersect_area'] = hc_intersect['geometry'].area

counties['county_area'] = counties['geometry'].area

hc_intersect = pd.merge(left = hc_intersect.drop(columns = ['geometry']), right = counties[['STATEFP', 'COUNTYFP', 'county_area']], how = 'left', on = ['STATEFP', 'COUNTYFP'])

hc_intersect['ratio'] = hc_intersect['intersect_area'] / hc_intersect['county_area']

#Export to file 
pd.DataFrame(hc_intersect.drop(columns = ['intersect_area', 'county_area']).rename(columns = {
    'STATEFP': 'STATE_ANSI', 'COUNTYFP': 'COUNTY_CODE', 'HUC6': 'HUC', 'ratio': 'proportion'
})).to_csv('Data/Selected/Watershed/geo_agg.csv', index = False)

In [ ]:
#Define function to take in dataframe aggregated by county
#and return dataframe aggregated by watershed
def watershed_agg(county_file, wagged_file):
    agg_rules = pd.read_csv(f"Data/Selected/Watershed/geo_agg.csv", dtype = 'str')
    wagged = pd.DataFrame()
    for huc in agg_rules['HUC'].unique():
        combine = pd.merge(left = pd.read_csv(county_file, dtype = 'str')[['STATE_ANSI', 'COUNTY_ANSI','YEAR', 'VALUE']].rename(columns = {'COUNTY_ANSI': 'COUNTY_CODE'}),
                 right = agg_rules[agg_rules['HUC'] == huc],
                 on = ['STATE_ANSI', 'COUNTY_CODE'],
                 how = 'inner')

        if combine.shape[0] > 0:
            combine['total'] = combine['VALUE'].astype('float') * combine['proportion'].astype('float')

            combine_totals = pd.DataFrame(combine.groupby('YEAR').sum()['total']).reset_index()

            combine_totals['HUC'] = huc
            wagged = pd.concat([wagged, combine_totals])

        wagged.to_csv(f'Data/Selected/Watershed/{wagged_file}.csv', index = False)

In [ ]:
#Aggregate all livestock head by watershed
watershed_agg('Data/Selected/Imputed/ChickensBroilersTimeImputed.csv','ChickensBroilers')
watershed_agg('Data/Selected/Imputed/CowsBeefTimeImputed.csv','CowsBeef')
watershed_agg('Data/Selected/Imputed/CowsMilkTimeImputed.csv','CowsMilk')
watershed_agg('Data/Selected/Imputed/HogsTimeImputed.csv','Hogs')
watershed_agg('Data/Selected/Imputed/ChickensLayersTimeImputed.csv', 'ChickensLayers')